In [ ]:
import torchlensmaker as tlm
import torch
import numpy as np
import matplotlib.pyplot as plt
import json

def sample_grid(xlim, ylim, N):
    x = np.linspace(xlim[0], xlim[1], N)
    y = np.linspace(ylim[0], ylim[1], N)
    X, Y = np.meshgrid(x, y)
    return torch.tensor(X), torch.tensor(Y), torch.tensor(np.stack((X, Y), axis=-1).reshape(-1, 2))


sag = tlm.XYPolynomial(torch.tensor([
    [0, 0, 0, 1],
    [1, 2.5, 0.5, 0],
    [0, -2.5, 1, 0],
]))

tau = torch.tensor(1.)

Y, Z, points = sample_grid([-1.0, 1.0], [-1.0, 1.0], 128)

G = sag.G(Y, Z, tau)
Gy, Gz = sag.G_grad(Y, Z, tau)

plt.figure()
plt.pcolormesh(Y, Z, G.reshape(Y.shape), cmap='RdBu_r', shading='auto')

plt.figure()
plt.pcolormesh(Y, Z, Gy.reshape(Y.shape), cmap='RdBu_r', shading='auto')

plt.figure()
plt.pcolormesh(Y, Z, Gz.reshape(Y.shape), cmap='RdBu_r', shading='auto')

surface = tlm.SagSurface(1., sag)

optics = tlm.Sequential(
    #tlm.PointSourceAtInfinity(0.9),
    tlm.Gap(2),
    tlm.RefractiveSurface(surface, material="water-nd"),
)

tlm.show3d(optics, sampling={"base": 256}, end=2)
#scene = tlm.render_sequence(optics, dim=3, sampling={"base": 64}, dtype=torch.float64)
#print(json.dumps(scene))

In [ ]:
import torchlensmaker as tlm
import torch
import numpy as np
import matplotlib.pyplot as plt
import json


def randC(*size):
    S = 0.2
    return S*(torch.rand(*size, dtype=torch.float64) - 0.5) * 2

surfaces = [
    tlm.SagSurface(10, tlm.XYPolynomial(randC(3, 4), normalize=True))
    for i in range(10)]

elements = []

for i, surface in enumerate(surfaces):
    elements.append(tlm.RefractiveSurface(surface, material=("water-nd" if i % 2 == 0 else "air")))
    elements.append(tlm.Gap(5))

optics = tlm.Sequential(
    tlm.PointSourceAtInfinity(6.0),
    tlm.Gap(5),
    *elements
)

tlm.show3d(optics, sampling={"base": 256}, end=2, controls={"show_optical_axis": True, "show_bounding_cylinders": True})

In [ ]:
## XYPolynomial optimization

import torchlensmaker as tlm
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import json


C = tlm.parameter(torch.tensor([[0.0, 0.0], [0.0, 0.0]], dtype=torch.float64))
#surface = tlm.SagSurface(10, tlm.XYPolynomial(C, normalize=True))

# works with SphereR
# nan in grad with Sphere
# nan in grad with XYPolynomial
# nan in grad with Parabola

#surface = tlm.SphereR(10, R=tlm.parameter(15))
surface = tlm.Parabola(10, tlm.parameter(0.02))

x = tlm.parameter(50.)

optics = tlm.Sequential(
    tlm.PointSourceAtInfinity(8.0),
    tlm.Gap(10),
    tlm.RefractiveSurface(surface, material="water-nd"),
    tlm.Gap(x),
    tlm.FocalPoint()
)

tlm.show3d(optics, sampling={"base": 100})

### get gradient
sampling = {"base": 100}
default_input = tlm.default_input(sampling, dim=3, dtype=torch.float64)
outputs = optics(default_input)
loss = outputs.loss
loss.backward()

params = list(surface.parameters().values())

print(surface.to_dict(3))
for param in params:
    print(param, param.grad)
    print()

print(x, x.grad)


tlm.show3d(optics, sampling={"base": 100})


In [ ]:
## XYPolynomial optimization

import torchlensmaker as tlm
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import json


C = tlm.parameter(torch.zeros((13,13), dtype=torch.float64))
curvature = tlm.parameter(torch.tensor(0.1690))

fixed_mask = torch.zeros_like(C, dtype=torch.bool)
fixed_mask[0, 0] = True  # Freeze position (0,0)

C.register_hook(lambda grad: grad.masked_fill(fixed_mask, 0.))

#surface = tlm.SagSurface(10, tlm.XYPolynomial(C, normalize=True))
#surface = tlm.Sphere(10, R=tlm.parameter(15))
surface = tlm.SagSurface(10, tlm.SagSum([
    #tlm.Spherical(C=tlm.parameter(torch.tensor(1/8.))),
    tlm.Spherical(C=curvature),
    #tlm.Parabolic(A=tlm.parameter(0.2), normalize=True),
    tlm.XYPolynomial(C, normalize=True)
]))

optics = tlm.Sequential(
    tlm.PointSourceAtInfinity(8.0),
    tlm.Gap(10),
    tlm.RefractiveSurface(surface, material="water-nd"),
    tlm.Gap(8),
    tlm.RefractiveSurface(surface, scale=-1., material="air"),
    tlm.Gap(15),
    tlm.FocalPoint()
)

tlm.show3d(optics, sampling={"base": 100})

tlm.optimize(
    optics,
    optimizer = tlm.optim.Adam(optics.parameters(), lr=2e-3),
    sampling = {"base": 10},
    dim = 3,
    num_iter = 100
).plot()

tlm.optimize(
    optics,
    optimizer = tlm.optim.Adam(optics.parameters(), lr=1e-4),
    sampling = {"base": 10},
    dim = 3,
    num_iter = 100
).plot()

tlm.optimize(
    optics,
    optimizer = tlm.optim.Adam(optics.parameters(), lr=1e-6),
    sampling = {"base": 10},
    dim = 3,
    num_iter = 500
).plot()

# print(surface.parameters())

tlm.show3d(optics, sampling={"base": 100})



In [ ]:
tlm.optimize(
    optics,
    optimizer = tlm.optim.Adam(optics.parameters(), lr=1e-4),
    sampling = {"base": 100},
    dim = 3,
    num_iter = 10
).plot()

tlm.show3d(optics, sampling={"base": 100})

print(C)
print(optics.parameters())

In [ ]:
## XYPolynomial optimization

import torchlensmaker as tlm
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import json

surface = tlm.Sphere(10, R=tlm.parameter(10))

optics = tlm.Sequential(
    tlm.PointSourceAtInfinity(8.0), 
    tlm.Gap(10),
    tlm.RefractiveSurface(surface, scale=-1., material="water-nd"),
    tlm.Gap(15),
    tlm.FocalPoint()
)

tlm.show3d(optics, sampling={"base": 10}, end=10, controls={"blocked_rays": "default", "show_bounding_cylinders": True})

In [ ]:
## XYPolynomial optimization

import torchlensmaker as tlm
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import json

#surface = tlm.Sphere(10, R=tlm.parameter(15))
#surface = tlm.Parabola(10, A=tlm.parameter(0.02))
surface = tlm.SagSurface(10, tlm.Parabolic(tlm.parameter(0.02)))

optics = tlm.Sequential(
    tlm.PointSourceAtInfinity(8.0),
    tlm.Gap(10),
    tlm.RefractiveSurface(surface, material="water-nd"),
    tlm.Gap(8),
    tlm.RefractiveSurface(surface, scale=-1., material="air"),
    tlm.Gap(15),
    tlm.FocalPoint()
)

tlm.show3d(optics, sampling={"base": 100})

torch.autograd.set_detect_anomaly(True)
tlm.optimize(
    optics,
    optimizer = tlm.optim.Adam(optics.parameters(), lr=2e-3),
    sampling = {"base": 10},
    dim = 3,
    num_iter = 10
).plot()

tlm.show3d(optics, sampling={"base": 100})

In [ ]:
torch.finfo(torch.float32).tiny